# Training Llama-2
This requires crazy amounts of memory so we will have to use LoRA and other tricks to make it fit on a gpu.

In [1]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 python-dotenv scipy tensorboardX tensorboard

In [6]:
from dotenv import load_dotenv

# load_dotenv()
load_dotenv("env.txt")

True

In [10]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [11]:
dataset = load_dataset(f"Roverto/ultimate-articles", token = os.environ["HF_TOKEN"])
dataset

Generating train split:   0%|          | 0/773 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'length'],
        num_rows: 773
    })
})

In [ ]:
model_name = "NousResearch/Llama-2-7b-hf"
dataset_name = "Roverto/ultimate-articles"
new_model_name = "llama-2-7b-ultimate-articles"

################################################################################
# QLoRA parameters
################################################################################
lora_r = 64 # LoRA attention dimension
lora_alpha = 16 # Alpha parameter for LoRA scaling
lora_dropout = 0.1 # Dropout probability for LoRA layers

################################################################################
# bitsandbytes parameters
################################################################################
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "./results" # where the model predictions and checkpoints will be stored
num_train_epochs = 1
fp16 = False # Enable f16 training
bf16 = True  # Enable bf16 training (set bf16 to True with an A100)
per_device_train_batch_size = 4 # Batch size per GPU for training
per_device_eval_batch_size = 4  # Batch size per GPU for evaluation
gradient_accumulation_steps = 1 # number of update steps to accumulate the gradients 
gradient_checkpointing = True
max_grad_norm = 0.3  # gradient clipping
learning_rate = 2e-4 # Initial learning rate (AdamW optimizer)
weight_decay = 0.001 # apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1         # (overrides num_train_epochs)
warmup_ratio = 0.03    # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = True # Group sequences into batches with same length
save_steps = 25        # Save checkpoint every X updates steps
logging_steps = 25     # Log every X updates steps

################################################################################
# SFT parameters
################################################################################
max_seq_length = None # Maximum sequence length to use
packing = False # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0} # Load the entire model on the GPU 0

In [14]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


Your GPU supports bfloat16: accelerate training with bf16=True


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

# Prepare data

In [15]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [16]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [22]:
context_length = 512
dataset = load_dataset(dataset_name, token = os.environ["HF_TOKEN"])
# dataset = dataset.map(
#     tokenize, 
#     batched=True, 
#     remove_columns=dataset["train"].column_names
# )
# dataset

# Load model with correct config

In [25]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


Map:   0%|          | 0/773 [00:00<?, ? examples/s]

In [31]:
# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.238300
50,2.275900
75,2.207100
100,2.162700
125,2.205100
150,2.212200
175,2.233400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=194, training_loss=2.212810811308241, metrics={'train_runtime': 326.5111, 'train_samples_per_second': 2.367, 'train_steps_per_second': 0.594, 'total_flos': 1.326549337030656e+16, 'train_loss': 2.212810811308241, 'epoch': 1.0})

# Try it out

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Run text generation pipeline with our next model
prompt = "how to avoid a hand block?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

In [32]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

# Save to Huggingface hub
This requires some trickery because it must be loaded in 16bit floats, not the 4-bit quantized model. The steps are:
1. Save model locally
2. Restart Kernel (to clear GPU memory)
3. Reload the base model in FP16 precision and use `peft` to merge everything

In [44]:
# # Save trained model
trainer.model.save_pretrained(new_model_name)

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 python-dotenv scipy tensorboardX tensorboard


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from dotenv import load_dotenv
# load_dotenv()
load_dotenv("env.txt")

model_name = "NousResearch/Llama-2-7b-hf"
dataset_name = "Roverto/ultimate-articles"
new_model_name = "llama-2-7b-ultimate-articles"

################################################################################
# QLoRA parameters
################################################################################
lora_r = 64 # LoRA attention dimension
lora_alpha = 16 # Alpha parameter for LoRA scaling
lora_dropout = 0.1 # Dropout probability for LoRA layers

################################################################################
# bitsandbytes parameters
################################################################################
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "./results" # where the model predictions and checkpoints will be stored
num_train_epochs = 1
fp16 = False # Enable f16 training
bf16 = True  # Enable bf16 training (set bf16 to True with an A100)
per_device_train_batch_size = 4 # Batch size per GPU for training
per_device_eval_batch_size = 4  # Batch size per GPU for evaluation
gradient_accumulation_steps = 1 # number of update steps to accumulate the gradients 
gradient_checkpointing = True
max_grad_norm = 0.3  # gradient clipping
learning_rate = 2e-4 # Initial learning rate (AdamW optimizer)
weight_decay = 0.001 # apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1         # (overrides num_train_epochs)
warmup_ratio = 0.03    # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = True # Group sequences into batches with same length
save_steps = 25        # Save checkpoint every X updates steps
logging_steps = 25     # Log every X updates steps

################################################################################
# SFT parameters
################################################################################
max_seq_length = None # Maximum sequence length to use
packing = False # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0} # Load the entire model on the GPU 0

In [4]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model_name)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model.push_to_hub(
    repo_id = new_model_name,
    private = True,
    token = os.environ["HF_TOKEN"]
)

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Roverto/llama-2-7b-ultimate-articles/commit/52b433938c6a10e7c382a0f14b6e9c141344f0dc', commit_message='Upload LlamaForCausalLM', commit_description='', oid='52b433938c6a10e7c382a0f14b6e9c141344f0dc', pr_url=None, pr_revision=None, pr_num=None)